In [1]:
from __future__ import print_function
from __future__ import division

import os
import math

import numpy as np
from astropy.io import fits
from astropy.table import Table
from scipy.stats import binned_statistic, scoreatpercentile
import pickle
from scipy.optimize import minimize
import time
from voronoi_2d_binning import voronoi_2d_binning
from sklearn.neighbors import NearestNeighbors
import pickle
import imp # reload modules if necessary

In [2]:
import binning
import bin_debiasing
import fit_debiasing
import make_dictionaries
import params

In [3]:
% matplotlib inline
import matplotlib as mpl
plt.rcParams['font.family'] = 'serif'
plt.rcParams['figure.figsize'] = (10.0, 8)
plt.rcParams['font.size'] = 18
mpl.ticker.AutoLocator.default_params['nbins'] = 5
mpl.ticker.AutoLocator.default_params['prune'] = 'both'

In [4]:
os.mkdir('output_files/') if os.path.isdir('output_files/') is False else None

source_directory = params.source_directory
full_sample = params.full_sample

min_log_fv = -1.5
max_log_fv = 0.01 

In [5]:
full_data = Table.read(source_directory + 'full_sample.fits')
print('Loaded galaxy data...')

Loaded galaxy data...


In [6]:
questions = make_dictionaries.questions
print('Loaded questions...')
function_dictionary = make_dictionaries.function_dictionary
print('Loaded functions...')

Loaded questions...
Loaded functions...


In [7]:
def reduce_sample(full_data,questions,question,p_cut=0.5,N_cut=5,normalised_values=True):
    ''' Reduce the sample to p>p_cut galaxies only, 
        using the precedeing questions'''
    
    previous_q = questions[question]['pre_questions']
    previous_a = questions[question]['pre_answers']
    
    if normalised_values == True:
        suffix = '_debiased_rh_normalised'
    else:
        suffix = '_debiased_rh'
    
    if previous_q != None:
        
        p_col = np.ones(len(full_data))
        
        for m in range(len(previous_q)):
            p_col = p_col*(full_data[previous_q[m] + '_' + previous_a[m] + suffix])
        N_col = (full_data[previous_q[-1] + '_' + previous_a[-1] + '_count'])
        
        select = (p_col > p_cut) & (N_col >= N_cut)
        data_reduced = full_data[select]
        print('{}/{} galaxies with p>{} and N>={}.'.format(len(data_reduced),
                                                          len(full_data),p_cut,N_cut))
    
    else:
        data_reduced = full_data.copy()
        print('Primary question, so all {} galaxies used.'.format(len(data_reduced)))
    
    return data_reduced

In [8]:
def get_bins(question,answer):
    ''' Get bins if they have already been created from a 
        previous running of the debiasing'''
    
    bins = Table.read('output_files/'+ question + '/' + answer + '/bins.fits')
    all_bins = Table.read('output_files/'+ question + '/' + answer + '/all_bins.fits')
    vbins_table = Table.read('output_files/'+ question + '/' + answer + '/vbin_parameters.fits')
    
    vbins = bins['vbin']
    zbins = bins['zbin']
    zbins_coarse = bins['coarse_zbin']
    vbins_all = all_bins['vbin']
    zbins_all = all_bins['zbin']
    zbins_coarse_all = all_bins['coarse_zbin']
    
    return vbins,zbins,zbins_coarse,vbins_all,zbins_all,zbins_coarse_all,vbins_table

In [9]:
def histogram_fractions(data,hist_bins):
    ''' Get raw histogram values '''
    h,bin_edges = np.histogram(data,bins=hist_bins)
    f = h/np.sum(h)
    return f


def bin_by_column(column, nbins, fixedcount=True):
    ''' Bin the data into redshift slices 
    (or by any column) '''
    
    sorted_indices = np.argsort(column)
    if fixedcount:
        bin_edges = np.linspace(0, 1, nbins + 1)
        bin_edges[-1] += 1
        values = np.empty(len(column))
        values[sorted_indices] = np.linspace(0, 1, len(column))
        bins = np.digitize(values, bins=bin_edges)
    else:
        bin_edges = np.linspace(np.min(column),np.max(column), nbins + 1)
        bin_edges[-1] += 1
        values = column
        bins = np.digitize(values, bins=bin_edges)
    x, b, n = binned_statistic(values, column, bins=bin_edges)
    return x, bins


def get_rms(dataset,reference,redshifts):
    ''' Calculate rms of a dataset in comparison w. reference data'''
    hist_bins = np.linspace(0,1,11)
    hist_bins[0] = -1
    hist_bins[-1] = 2
    zv,zb = bin_by_column(redshifts,nbins=10)
    
    rms_values = np.zeros(len(np.unique(zb)))
    ref_fractions = histogram_fractions(reference,hist_bins)
    
    for i,z in enumerate(np.unique(zb)):
        
        sample = dataset[zb == z]
        fractions = histogram_fractions(sample,hist_bins)
        rms_values[i] = np.sum((fractions-ref_fractions)**2)
    
    return np.sum(rms_values)

In [10]:
def choose_best_function(raw_data,debiased,question,answer):
    ''' Decide on which set of debiasing values is preferred'''
    volume_ok = raw_data['in_volume_limit'] == 1
    
    # Raw data for reference:
    vl  = raw_data[volume_ok][question + '_' + answer + '_weighted_fraction']
    
    # 2 sets of debiased data for comparison:
    vl_bin = debiased['bin_method'][volume_ok]
    vl_fit = debiased['fit_method'][volume_ok]
    redshifts = full_data['REDSHIFT_1'][volume_ok]
    
    low_z = (redshifts >= 0.03) & (redshifts < 0.035)
    reference = vl[low_z] # raw low-z for comparison
    
    rms_bin = get_rms(vl_bin,reference,redshifts) # get rms values
    rms_fit = get_rms(vl_fit,reference,redshifts) # get rms values
    
    print('rms^2(bin) = {0:.3f}'.format(rms_bin))
    print('rms^2(fit) = {0:.3f}'.format(rms_fit))
    if rms_bin < rms_fit:
        print('---> bin method selected')
        debiased_values = debiased['bin_method']
    else:
        print('---> fit method selected')
        debiased_values = debiased['fit_method'] # Prefer fit method if it 
        # has better rms^2 value.
        
    return debiased_values

In [11]:
def bin_and_debias(full_data,question,questions,answer
                   ,bins_exist=False,n_per_bin=100,coarse=False):
    '''Set to 'coarse' to make the fitting only apply to the 'coarse binning'of 4 redshift bins per 
    voronoi bin rather than the fully binned data'''
    
    # Create output file folders:
    (os.mkdir('output_files/'+ question) if
     os.path.isdir('output_files/'+ question) is False else None)
    (os.mkdir('output_files/'+ question + '/' + answer) if
     os.path.isdir('output_files/'+ question + '/' + answer) is False else None)
    
    # Only use the p>0.5 and N>= 5 data for a given question:
    data = reduce_sample(full_data,questions,question)
    
    if bins_exist == True:
        (vbins,zbins,zbins_coarse,
         vbins_all,zbins_all,zbins_coarse_all,
         vbins_table) = get_bins(question,answer)
        print('Bins obtained from previous iteration...')
        
    else:
        (vbins,zbins,zbins_coarse,
         vbins_all,zbins_all,zbins_coarse_all,
         vbins_table) = binning.bin_data(data,full_data,
                                        question,answer,
                                        signal=n_per_bin)
        
    # Save the binning data:
    bin_table = Table([vbins,zbins,zbins_coarse],names=('vbin','zbin','coarse_zbin'))
    all_bin_table = Table([vbins_all,zbins_all,zbins_coarse_all],names=('vbin','zbin','coarse_zbin'))
    bin_table.write('output_files/'+ question + '/' + answer + '/bins.fits',overwrite=True)
    all_bin_table.write('output_files/'+ question + '/' + answer + '/all_bins.fits',overwrite=True)
    vbins_table.write('output_files/'+ question + '/' + answer + '/vbin_parameters.fits',overwrite=True)

    # Debias using the discrete bin method 1st:
    debiased_bin = bin_debiasing.debias(data,full_data,vbins,zbins,vbins_all,zbins_all,question,answer)
    # Now debias using the functional fitting method:
    debiased_fit,dout,fit_setup,zbins,fit_vbin_results = fit_debiasing.debias_by_fit(data,full_data,vbins,zbins,
                                                                                     zbins_coarse,question,answer,
                                                                                     function_dictionary,min_log_fv,
                                                                                     coarse=coarse)
    
    volume_ok = data['in_volume_limit'] == 1    
    vl_data = full_data[volume_ok]
    vl_fit = debiased_fit[volume_ok]
    vl_bin = debiased_bin[volume_ok]

    debiased_table = Table([debiased_bin,debiased_fit],names=('bin_method','fit_method'))
    debiased_table.write('output_files/'+ question + '/' + answer + '/debiased.fits',overwrite=True)
    dout.write('output_files/'+ question + '/' + answer + '/fit_results.fits',overwrite=True)
    pickle.dump(fit_setup,open('output_files/'+ question + '/' + answer + '/fit_setup.p', "wb" ))
    
    return debiased_table

In [13]:
question_order = ['t01_smooth_or_features'
                  ,'t02_edgeon'
                  ,'t07_rounded'
                  ,'t03_bar'
                  ,'t04_spiral'
                  ,'t09_bulge_shape'
                  ,'t05_bulge_prominence'
                  ,'t10_arms_winding'
                  ,'t11_arms_number'
                  ,'t06_odd'
                  ,'t08_odd_feature']

for question in question_order:
    answers = questions[question]['answers']
    
    for answer in answers:
        
        #bins_exist = os.path.isfile('output_files/'+ question + '/' + answer + '/bins.fits')
        bins_exist = False
        
        print('----------------------------------')
        print('Question to be debiased:',question)
        print('Answer to be debiased:',answer)
        
        debiased = bin_and_debias(full_data,question,questions,answer,
                                  bins_exist=bins_exist,n_per_bin=50,coarse=False) # Use smaller bins!
        
        deb_vals = choose_best_function(full_data,debiased,question,answer)
        full_data[question + '_' + answer + '_debiased_rh'] = deb_vals
        
        print('----------------------------------')

    debiased_values = np.array([full_data[question + '_' + a + '_debiased_rh'] for a in answers])
    debiased_norm = debiased_values/np.sum(debiased_values,axis=0)
    debiased_norm[np.isnan(debiased_norm)] = 0
    for m in range(len(debiased_norm)):
        full_data[question + '_' + answers[m] + '_debiased_rh_normalised'] = debiased_norm[m]

----------------------------------
Question to be debiased: t08_odd_feature
Answer to be debiased: a19_ring
33591/228201 galaxies with p>0.5 and N>=5.
Bin-accretion...
182  initial bins.
Reassign bad bins...
25  good bins.
Modified Lloyd algorithm...
17  iterations.
Unbinned pixels:  0  /  2386
Fractional S/N scatter (%): 9.75702862927
25 voronoi bins
10.48 redshift bins per voronoi bin
All bins fitted! 6.208342790603638s in total
chisq(logistic) = 0.002256487463289655
All bins fitted! 4.781087160110474s in total
chisq(exp. power) = 0.0001024757570263222
All bins fitted! 11.321000099182129s in total
Selected functions:------
k: linear(M),log(R),log(z)
c: linear(M),log(R),exp(z)
rms^2(bin) = 2.287
rms^2(fit) = 0.641
---> fit method selected
----------------------------------
----------------------------------
Question to be debiased: t08_odd_feature
Answer to be debiased: a20_lens_or_arc
33591/228201 galaxies with p>0.5 and N>=5.
Bin-accretion...
199  initial bins.
Reassign bad bins...


/home/ppxrh/anaconda3/lib/python3.4/site-packages/ipykernel/__main__.py:50: RuntimeWarning: invalid value encountered in true_divide


Save the debiased values to a FITS file, which can be combined in to full sample using Recover_full_data.ipynb

In [14]:
debiased_columns = []

for m in range(len(full_data.colnames)):
    c = full_data.colnames[m]
    if 'debiased_rh' in c:
        debiased_columns.append(c)
        
debiased_values = full_data[debiased_columns]
debiased_values.write(source_directory + 'debiased_values.fits',overwrite=True)